# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [9]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_omission"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240227_130241_comp_om_subj_4-2_and_4-3,"[1, 242]","[[1, 242], [242, 261], [628, 644], [644, 648],...","[[1, 242], [44043, 44055], [44696, 44699], [44...",20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,"[0, 193420]","[[0, 193420], [193820, 208217], [504224, 51562...","[[0, 193420], [35331875, 35342275], [35856482,..."
0,20240227_130241_comp_om_subj_4-2_and_4-3,"[1737, 1988]","[[1, 242], [242, 261], [628, 644], [644, 648],...","[[1, 242], [44043, 44055], [44696, 44699], [44...",20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,"[1393833, 1593838]","[[0, 193420], [193820, 208217], [504224, 51562...","[[0, 193420], [35331875, 35342275], [35856482,..."
0,20240227_130241_comp_om_subj_4-2_and_4-3,"[4730, 4978]","[[1, 242], [242, 261], [628, 644], [644, 648],...","[[1, 242], [44043, 44055], [44696, 44699], [44...",20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,"[3793867, 3993867]","[[0, 193420], [193820, 208217], [504224, 51562...","[[0, 193420], [35331875, 35342275], [35856482,..."
0,20240227_130241_comp_om_subj_4-2_and_4-3,"[7597, 7846]","[[1, 242], [242, 261], [628, 644], [644, 648],...","[[1, 242], [44043, 44055], [44696, 44699], [44...",20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,"[6093895, 6293897]","[[0, 193420], [193820, 208217], [504224, 51562...","[[0, 193420], [35331875, 35342275], [35856482,..."
0,20240227_130241_comp_om_subj_4-2_and_4-3,"[9841, 10090]","[[1, 242], [242, 261], [628, 644], [644, 648],...","[[1, 242], [44043, 44055], [44696, 44699], [44...",20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,"[7893918, 8093920]","[[0, 193420], [193820, 208217], [504224, 51562...","[[0, 193420], [35331875, 35342275], [35856482,..."
...,...,...,...,...,...,...,...,...,...,...,...
33,20240325_150329_comp_om_subj_4-2_and_4-4,"[55587, 55786]","[[1, 112], [141, 239], [241, 246], [248, 281],...","[[1, 112], [31583, 31585], [31585, 31588], [31...",20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,"[55612954, 55812956]","[[0, 111862], [140662, 239263], [241066, 24606...","[[0, 111862], [31564053, 31566453], [31566856,..."
33,20240325_150329_comp_om_subj_4-2_and_4-4,"[56585, 56784]","[[1, 112], [141, 239], [241, 246], [248, 281],...","[[1, 112], [31583, 31585], [31585, 31588], [31...",20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,"[56612966, 56812969]","[[0, 111862], [140662, 239263], [241066, 24606...","[[0, 111862], [31564053, 31566453], [31566856,..."
33,20240325_150329_comp_om_subj_4-2_and_4-4,"[57983, 58182]","[[1, 112], [141, 239], [241, 246], [248, 281],...","[[1, 112], [31583, 31585], [31585, 31588], [31...",20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,"[58012981, 58212983]","[[0, 111862], [140662, 239263], [241066, 24606...","[[0, 111862], [31564053, 31566453], [31566856,..."
33,20240325_150329_comp_om_subj_4-2_and_4-4,"[60178, 60378]","[[1, 112], [141, 239], [241, 246], [248, 281],...","[[1, 112], [31583, 31585], [31585, 31588], [31...",20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,"[60213011, 60413011]","[[0, 111862], [140662, 239263], [241066, 24606...","[[0, 111862], [31564053, 31566453], [31566856,..."


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/{}_per_video_trial_labels.pkl".format(OUTPUT_PREFIX))

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240227_130241_comp_om_subj_4-2_and_4-3,1,242,[ 1 242],[ 1 242],20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,0,193420,[ 0 193420],[ 0 193420]
0,20240227_130241_comp_om_subj_4-2_and_4-3,1737,1988,[1883 1885] [1885 1895] [1895 1905] [1907 1915...,,20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,1393833,1593838,[1509237 1510837] [1511837 1519237] [1520037 1...,
0,20240227_130241_comp_om_subj_4-2_and_4-3,4730,4978,[4944 4985],,20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,3793867,3993867,[3965869 3998470],
0,20240227_130241_comp_om_subj_4-2_and_4-3,7597,7846,[7767 7768] [7770 7774] [7777 7787] [7787 7789...,,20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,6093895,6293897,[6229499 6231496] [6232499 6235899] [6238099 6...,
0,20240227_130241_comp_om_subj_4-2_and_4-3,9841,10090,[9841 9843] [9845 9847] [9848 9855] [9857 9873...,,20240227_130241_comp_om_subj_4-2_and_4-3.1.vid...,"[4.2, 4.3]",1981794,70466080,7893918,8093920,[7895120 7896118] [7897121 7898518] [7899521 7...,


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
33,20240325_150329_comp_om_subj_4-2_and_4-4,55587,55786,[55566 55613] [55625 55625] [55635 55643] [556...,[55585 55588] [55593 55595] [55598 55610] [556...,20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,55612954,55812956,[55592553 55638754] [55651154 55651952] [55660...,[55610953 55614353] [55619153 55620551] [55623...
33,20240325_150329_comp_om_subj_4-2_and_4-4,56585,56784,[56584 56590] [56590 56591] [56592 56598] [566...,[56632 56644] [56692 56693] [56704 56707] [567...,20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,56612966,56812969,[56612366 56617364] [56617764 56619166] [56619...,[56660964 56671167] [56720170 56721364] [56732...
33,20240325_150329_comp_om_subj_4-2_and_4-4,57983,58182,[57993 58014] [58016 58022] [58037 58037] [580...,[58009 58022] [58022 58029] [58029 58031] [580...,20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,58012981,58212983,[58023781 58045184] [58046184 58053381] [58066...,[58039981 58053381] [58053781 58060584] [58060...
33,20240325_150329_comp_om_subj_4-2_and_4-4,60178,60378,[60172 60204] [60215 60226] [60250 60251] [602...,[60176 60186] [60187 60189] [60189 60190] [601...,20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,60213011,60413011,[60206410 60238608] [60250611 60261211] [60284...,[60210611 60221208] [60222413 60223208] [60223...
33,20240325_150329_comp_om_subj_4-2_and_4-4,61276,61475,[61279 61284] [61290 61295] [61295 61297] [612...,[61453 61453] [61455 61456] [61456 61460] [614...,20240325_150329_comp_om_subj_4-2_and_4-4.2.vid...,"[4.2, 4.4]",2013669,65819102,61313024,61513024,[61315824 61321422] [61326221 61332222] [61333...,[61489824 61491024] [61491624 61493227] [61493...


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/{}_per_video_trial_labels.csv".format(OUTPUT_PREFIX))